In [7]:
import pandas as pd
import os
import scipy.stats
from yfetch import get_stock_history

symbol_map = {}
symbol_types = {} # symbol : ETF/stock

# sp500_stocks = pd.read_csv('data/sp500_stocks.csv')
# symbol_map.update(sp500_stocks.set_index('Symbol')['Security'].to_dict())
# symbol_types.update({symbol: 'stock' for symbol in sp500_stocks['Symbol']})

# top_etfs = pd.read_csv('data/top-etfs.csv')
# symbol_map.update(top_etfs.set_index('Symbol')['ETF Name'].to_dict())
# symbol_types.update({symbol: 'ETF' for symbol in top_etfs['Symbol']})

# symbol_map = {k.replace('.', '-'): v for k, v in symbol_map.items()}

ucits = {
  'EHF1.DE': 'Amundi MSCI Europe High Dividend Factor UCITS',
  'EHDV.DE': 'Invesco EURO STOXX High Dividend Low Volatility UCITS',
  'QDVE.DE': 'iShares S&P 500 Information Technology Sector UCITS',
  'SPPY.DE': 'SPDR S&P 500 Leaders UCITS',
  'SPPE.DE': 'SPDR S&P 500 UCITS',
  'NQSE.DE': 'iShares VII PLC - iShares NASDAQ 100 UCITS',
  'EXV1.DE': 'iShares STOXX Europe 600 Banks UCITS',
  'VDIV.DE': 'VanEck Morningstar Developed Markets Dividend Leaders UCITS', # not on Revolut

  'SPYZ.DE': 'SPDR MSCI Europe Financials UCITS',
  'EXH2.DE': 'iShares STOXX Europe 600 Financial Services UCITS',
  'ESIF.DE': 'iShares MSCI Europe Financials Sector UCITS',
  'SC02.DE': 'Invesco STOXX Europe 600 Optimised Financial Services UCITS',
}
symbol_map.update(ucits)
symbol_types.update({symbol: 'ETF' for symbol in ucits.keys()})

In [8]:
window = 13 # weeks
# min_history = 261  # 5 years of weekly data
min_history = 104  # 2 years of weekly data
rows = []

directory = 'data/stock_history'
for symbol, description in symbol_map.items():
  print(f'[{len(rows)+1}/{len(symbol_map)}] Processing {symbol} {symbol_types[symbol]} ({description})')
  history = get_stock_history(symbol, interval='1wk', cache_days=30)
  if len(history) < min_history:
    print(f'Skipping {symbol}: only {len(history)} weeks of data')
    continue
  history = history.tail(min_history)
  sma = history.Close.rolling(window=13).mean().dropna()
  cagr = (sma.iloc[-1] / sma.iloc[0]) ** (1 / (len(sma) / 52)) - 1
  changes = history.Close.pct_change(periods=window).dropna()
  median = changes.median()
  gmean_change = scipy.stats.gmean(1 + changes) - 1  # Geometric mean of changes
  std = changes.std()
  fields = {
      'symbol': symbol,
      'description': description,
      'type': symbol_types[symbol],
      'weeks': len(history),
      'cagr': cagr,
      'median change': median,
      'gmean change': gmean_change,
      'std': std,
      'gmean/std': std and gmean_change / std,
      'median/std': std and median / std
  }
  rows.append(fields)

df = pd.DataFrame(rows).sort_values(by='median/std', ascending=False)
df.to_csv(f'data/top-stox-{min_history}w.csv', index=False)

[1/12] Processing EHF1.DE ETF (Amundi MSCI Europe High Dividend Factor UCITS)
[2/12] Processing EHDV.DE ETF (Invesco EURO STOXX High Dividend Low Volatility UCITS)
[3/12] Processing QDVE.DE ETF (iShares S&P 500 Information Technology Sector UCITS)
[4/12] Processing SPPY.DE ETF (SPDR S&P 500 Leaders UCITS)
[5/12] Processing SPPE.DE ETF (SPDR S&P 500 UCITS)
[6/12] Processing NQSE.DE ETF (iShares VII PLC - iShares NASDAQ 100 UCITS)
[7/12] Processing EXV1.DE ETF (iShares STOXX Europe 600 Banks UCITS)
[8/12] Processing VDIV.DE ETF (VanEck Morningstar Developed Markets Dividend Leaders UCITS)
[9/12] Processing SPYZ.DE ETF (SPDR MSCI Europe Financials UCITS)
[10/12] Processing EXH2.DE ETF (iShares STOXX Europe 600 Financial Services UCITS)
[11/12] Processing ESIF.DE ETF (iShares MSCI Europe Financials Sector UCITS)
[12/12] Processing SC02.DE ETF (Invesco STOXX Europe 600 Optimised Financial Services UCITS)


In [9]:
for col in ['cagr', 'gmean change', 'median change', 'std']:
    df[col] = df[col].map('{:.2%}'.format)
display(df)

,symbol,description,type,weeks,cagr,median change,gmean change,std,gmean/std,median/std
10,ESIF.DE,iShares MSCI Europe Financials Sector UCITS,ETF,104,35.97%,8.06%,8.08%,4.73%,1.709428,1.706006
8,SPYZ.DE,SPDR MSCI Europe Financials UCITS,ETF,104,35.97%,7.91%,8.08%,4.78%,1.690546,1.655074
7,VDIV.DE,VanEck Morningstar Developed Markets Dividend ...,ETF,104,18.87%,5.34%,4.47%,3.74%,1.194037,1.427653
0,EHF1.DE,Amundi MSCI Europe High Dividend Factor UCITS,ETF,104,15.40%,3.70%,3.69%,3.22%,1.146310,1.148794
6,EXV1.DE,iShares STOXX Europe 600 Banks UCITS,ETF,104,48.90%,8.69%,10.57%,7.78%,1.358140,1.116346
1,EHDV.DE,Invesco EURO STOXX High Dividend Low Volatilit...,ETF,104,24.63%,5.64%,5.73%,5.17%,1.108393,1.089952
11,SC02.DE,Invesco STOXX Europe 600 Optimised Financial S...,ETF,104,21.98%,5.65%,5.17%,5.36%,0.965759,1.054598
9,EXH2.DE,iShares STOXX Europe 600 Financial Services UCITS,ETF,104,21.34%,4.96%,5.04%,5.52%,0.913079,0.899312
4,SPPE.DE,SPDR S&P 500 UCITS,ETF,104,21.90%,5.91%,5.14%,6.69%,0.768257,0.883840
5,NQSE.DE,iShares VII PLC - iShares NASDAQ 100 UCITS,ETF,104,24.38%,7.30%,5.68%,8.64%,0.657632,0.844798
